# LSTM Remaining Useful Life (RUL) Model Pipeline
This notebook builds a complete pipeline for training an LSTM model on the CMAPSS FD002 dataset.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('train_FD002.csv')
df.head()

In [ ]:
# Compute RUL
max_cycle = df.groupby('unit')['cycle'].max().reset_index().rename(columns={'cycle': 'max_cycle'})
max_cycle
#df = df.merge(max_cycle, on='unit', how='left')

#df['RUL'] = df['max_cycle'] - df['cycle']
#df[['unit','cycle','RUL']].head()


In [ ]:
# Feature selection
exclude = {'unit','cycle','RUL','max_cycle'}
feature_cols = [c for c in df.columns if c not in exclude]
features = df[feature_cols].values
labels = df['RUL'].values

# Scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
scaled_df = pd.DataFrame(scaled_features, columns=feature_cols)
scaled_df['RUL'] = labels
scaled_df['unit'] = df['unit']
scaled_df['cycle'] = df['cycle']
scaled_df.head()

In [ ]:
# Sequence building
TIME_STEPS = 50
X_list, y_list = [], []
for uid, group in scaled_df.groupby('unit'):
    g = group[feature_cols + ['RUL']].values
    if len(g) >= TIME_STEPS:
        for i in range(len(g) - TIME_STEPS + 1):
            X_list.append(g[i:i+TIME_STEPS, :-1])  # sensors
            y_list.append(g[i+TIME_STEPS-1, -1])   # RUL at end

X = np.array(X_list)
y = np.array(y_list)
X.shape, y.shape

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

In [ ]:
# Build LSTM model
tf.keras.backend.clear_session()
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(loss='mse', optimizer=Adam(1e-3), metrics=['mae'])
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    verbose=1
)

In [ ]:
pred = model.predict(X_test).reshape(-1)
y_true = y_test.reshape(-1)

print("Pred shape:", pred.shape)
print("True shape:", y_true.shape)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Training & Validation Loss (FD002)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
N = 200

plt.figure(figsize=(10,4))
plt.plot(y_true[:N], label="True RUL")
plt.plot(pred[:N], label="Predicted RUL")
plt.title("True vs Predicted RUL (First 200 Samples) - FD002")
plt.xlabel("Sample Index")
plt.ylabel("RUL")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(5.5,5.5))
plt.scatter(y_true, pred, alpha=0.35)

lo = min(y_true.min(), pred.min())
hi = max(y_true.max(), pred.max())

plt.plot([lo, hi], [lo, hi], "--")
plt.title("Predicted vs True RUL - FD002")
plt.xlabel("True RUL")
plt.ylabel("Predicted RUL")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# -------- 7. evaluate --------
# Typical MAE for FD001 using a basic LSTM:
# 	•	MAE < 20 → Excellent
# 	•	MAE 20–30 → Good
# 	•	MAE 30–50 → Acceptable
# 	•	MAE > 50 → Poor model performance
mae = np.mean(np.abs(y_true - pred))
rmse = np.sqrt(np.mean((y_true - pred)**2))
mse = np.mean((y_true - pred)**2)

print("\n=== FD002 Test Metrics ===")
print(f"MAE : {mae:.3f}")
print(f"MSE : {mse:.3f}")
print(f"RMSE: {rmse:.3f}")


In [ ]:
# #1 Correlation with RUL (Fast Diagnostic)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sensor_cols = [c for c in df.columns if "sensor" in c]

corrs = df[sensor_cols + ["RUL"]].corr()["RUL"].abs().sort_values(ascending=False)

plt.figure(figsize=(10,10))
sns.barplot(x=corrs.values, y=corrs.index)
plt.title("Sensor–RUL Correlation Importance")
plt.xlabel("Absolute Correlation with RUL")
plt.ylabel("Sensor")
plt.show()

print(corrs)

In [ ]:
#1 Correlation with RUL (Fast Diagnostic)
corr = df.corr()['RUL'].abs().sort_values(ascending=False)
corr.head(15)

In [ ]:
# 2: Time Series Stability / Degradation Analysis

# Some sensors don’t change at all (flat line) → useless
# Some sensors vary but not related to failure → noise
# Some sensors show consistent degradation → extremely important

# Identify sensors with clear downward or upward trends

import numpy as np

slopes = {}

for s in sensor_cols:
    slopes[s] = np.polyfit(df["cycle"], df[s], 1)[0]

pd.Series(slopes).sort_values()

	# •	Large negative slope = strong degradation
	# •	Large positive slope = increasing stress/temp etc → can also be important
	# •	Values near zero = poor signal

In [ ]:
# 3. Permutation Importance
import numpy as np
from sklearn.metrics import mean_absolute_error

def permutation_importance_lstm(model, X_val, y_val, feature_names):
    baseline_pred = model.predict(X_val)
    baseline_mae = mean_absolute_error(y_val, baseline_pred)

    importance = {}

    for i, name in enumerate(feature_names):
        X_shuffled = X_val.copy()

        # shuffle only one feature across all time steps
        X_shuffled[:,:,i] = np.random.permutation(X_shuffled[:,:,i].flatten()).reshape(X_val[:,:,i].shape)

        pred = model.predict(X_shuffled)
        mae = mean_absolute_error(y_val, pred)
        importance[name] = mae - baseline_mae

    return sorted(importance.items(), key=lambda x: x[1], reverse=True)



important_sensors = permutation_importance_lstm(model, X_test, y_test, feature_cols)
important_sensors[:10]

In [ ]:
#4.PCA / Health Indicator Contribution
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(df[feature_cols])

importance = pd.Series(np.abs(pca.components_[0]), index=feature_cols)
importance.sort_values(ascending=False).head(10)

In [ ]:
key_sensors = ["sensor_9", "sensor_18", "sensor_8", "sensor_12", "sensor_14", "sensor_7", "sensor_3", "sensor_4", "sensor_13", "sensor_2"]

In [ ]:
#
weights = {
    'sensor_9':	0.712347,
'sensor_18':	0.286250,
'sensor_8':	0.286094,
'sensor_7':	0.280184,
'sensor_12':	0.264340,
'sensor_4':	0.248423,
'sensor_3':	0.224224,
'sensor_13':	0.185068,
'sensor_14':	0.144903,
'sensor_2':	0.076542
}

df["CDI_weighted"] = sum(df[s] * w for s, w in weights.items())

In [ ]:
df_norm = (df[key_sensors] - df[key_sensors].min()) / (df[key_sensors].max() - df[key_sensors].min())
df["HI_norm"] = df_norm.mean(axis=1)
df_norm

In [ ]:
# PCA-Based Health Index
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = df[key_sensors]

# Standardize (PCA works best on standardized data)
scaler_hi = StandardScaler()
X_scaled = scaler_hi.fit_transform(X)

# PCA
pca = PCA(n_components=1)
df["HI_pca"] = pca.fit_transform(X_scaled)

df["HI_pca"]

In [ ]:
import matplotlib.pyplot as plt

unit_id = 8
g = df[df["unit"] == unit_id]

plt.figure(figsize=(8,6))
#plt.plot(g["cycle"], g["HI_pca"], label="PCA Health Index")
#plt.plot(g["cycle"], g["CDI_weighted"], label="Weighted CDI", alpha=0.7)
plt.plot(g["cycle"], g["HI_norm"], label="Normalized HI", alpha=0.7)

plt.gca().invert_yaxis()        # health decreases over time
plt.title(f"Health Index for Engine Unit {unit_id}")
plt.xlabel("Cycle")
plt.ylabel("Health Indicator")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

def fft_energy(x):
    """
    Sum of squared magnitudes of the FFT (a simple measure of signal power).
    x is a 1D numpy array (rolling window of one sensor).
    """
    # real FFT (only non-negative frequencies)
    f = np.fft.rfft(x)
    mag = np.abs(f)
    return float(np.sum(mag ** 2))

def fft_peak(x):
    """
    Maximum magnitude in the FFT spectrum (dominant amplitude).
    """
    f = np.fft.rfft(x)
    mag = np.abs(f)
    return float(np.max(mag))

In [ ]:
# Important sensors (from your feature importance results)
fft_sensors = ["sensor_9", "sensor_18", "sensor_8", "sensor_12", "sensor_13", "sensor_7", "sensor_3", "sensor_4", "sensor_14", "sensor_2"]

# Rolling window (in cycles) for FFT computation
FFT_WINDOW = 50  # you can try 30, 50, 100 etc.

In [ ]:
# We'll compute FFT features per engine (unit) over a rolling window
for col in fft_sensors:
    energy_col = f"{col}_fft_energy"
    peak_col   = f"{col}_fft_peak"

    # Rolling by engine unit
    df[energy_col] = (
        df.groupby("unit")[col]
          .rolling(window=FFT_WINDOW, min_periods=FFT_WINDOW)
          .apply(fft_energy, raw=True)
          .reset_index(level=0, drop=True)
    )

    df[peak_col] = (
        df.groupby("unit")[col]
          .rolling(window=FFT_WINDOW, min_periods=FFT_WINDOW)
          .apply(fft_peak, raw=True)
          .reset_index(level=0, drop=True)
    )

# Optional: check new columns
#df[[ "unit", "cycle" ] + [s + "_fft_energy" for s in fft_sensors]].head(60)

In [ ]:
unit_id = 4  # choose any engine ID that exists in your data
plot_cols = ["sensor_9", "sensor_9_fft_energy", "sensor_9_fft_peak"]

tmp = df[df["unit"] == unit_id].reset_index(drop=True)

ax = tmp.plot(x="cycle", y="sensor_9", label="sensor_9 (time)", figsize=(10,6))
ax2 = ax.twinx()
tmp.plot(x="cycle", y="sensor_9_fft_energy", label="sensor_9_fft_energy", ax=ax2, style='r--')
tmp.plot(x="cycle", y="sensor_9_fft_peak", label="sensor_9_fft_peak", ax=ax2, style='g:')

ax.set_ylabel("Raw sensor_9")
ax2.set_ylabel("FFT features")
ax.set_title(f"Engine {unit_id}: sensor_9 & FFT features")
ax.legend(loc="upper left")
ax2.legend(loc="upper right")